<a href="https://colab.research.google.com/github/jantuitman/App5/blob/master/DwProfessionsEducations/Professions_Educations_Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install faiss-cpu
!pip install OpenAI
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 66.3 MB/s eta 0:00:00


In [8]:
import yaml

def read_secrets():
    with open("drive/MyDrive/InputData/secrets.yml", 'r') as file:
        data = yaml.safe_load(file)
        return data

secrets = read_secrets()
api_key = secrets['open-api']['key']


In [10]:

import openai
def get_openai_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model,api_key=api_key)['data'][0]['embedding']

In [55]:
import faiss
import numpy as np
import copy
import os
import pickle

class EducationIndex:
    def __init__(self, dim=1536):
        self.dim = dim
        self.index = faiss.IndexFlatIP(self.dim)
        self.education_data = {}  # To store the original data fields
        self.counter = 0  # To keep track of the number of added educations

    def get_embedding(self, text):
        # TODO: Implement the actual embedding logic using OpenAI's API
        # For the purpose of this example, we'll return a random vector
        return get_openai_embedding(text)

    def add_education(self, education_entry):
        studiebeschrijving = education_entry.get('Studiebeschrijving')
        naamopleiding = education_entry.get('NaamOpleiding')
        soortopleiding = education_entry.get('SoortOpleiding')
        soortho = education_entry.get('SoortHo')

        if studiebeschrijving and not pd.isna(studiebeschrijving):
            # Generate the title
            title = f"{naamopleiding} ({soortopleiding}, {soortho})"

            # Concatenate the title and description
            full_text = title + " - " + str(studiebeschrijving)

            # Get the embedding
            embedding = self.get_embedding(full_text)

            self.index.add(np.array([embedding]))
            self.education_data[self.counter] = education_entry
            self.counter += 1



    def search(self, query_text, k=5):
        query_embedding = self.get_embedding(query_text)
        D, I = self.index.search(np.array([query_embedding]), k)
        results = []
        for idx, score in zip(I[0], D[0]):
          if idx != -1:
            result = copy.deepcopy(self.education_data[idx])
            result['score'] = score
            results.append(result)
        return results

    def search_by_vector(self, query_vector, k=5):
        D, I = self.index.search(np.array([query_vector]), k)
        results = []
        for idx, score in zip(I[0], D[0]):
            if idx != -1:
                result = copy.deepcopy(self.education_data[idx])
                result['score'] = score
                results.append(result)
        return results

    def save(self, directory_path):
        # Check if the directory exists; create it if it doesn't
        if not os.path.exists(directory_path):
            try:
                os.makedirs(directory_path)
            except Exception as e:
                print(f"Could not create directory: {e}")
                return

        # Construct the file paths
        faiss_file_path = os.path.join(directory_path, 'faiss_index.idx')
        data_file_path = os.path.join(directory_path, 'education_data.pkl')

        # Save the FAISS index
        faiss.write_index(self.index, faiss_file_path)

        # Save the education_data dictionary and counter
        with open(data_file_path, 'wb') as f:
            pickle.dump((self.education_data, self.counter), f)


    def load(self, directory_path):
        # Construct the file paths
        faiss_file_path = os.path.join(directory_path, 'faiss_index.idx')
        data_file_path = os.path.join(directory_path, 'education_data.pkl')

        # Load the FAISS index
        self.index = faiss.read_index(faiss_file_path)

        # Load the education_data dictionary and counter
        with open(data_file_path, 'rb') as f:
            self.education_data, self.counter = pickle.load(f)


# Example usage
edu_index2 = EducationIndex()

# Adding an example education (you would loop through your actual data)
example_education = {
    'OpleidingscodeSamen': 30105,
    'NaamOpleiding': 'Security Management',
    'SoortOpleiding': 'Bachelor',
    'SoortHo': 'HBO',
    'Studiebeschrijving': 'De opleiding security management leert belangrijke skills over criminologie en andere vaardigheden, benodigd voor het politie vak of voor beveiliging'
}
edu_index2.add_education(example_education)




In [26]:
edu_index.search('Een politieagent moet veel weten over criminologie')

[{'OpleidingscodeSamen': 30105,
  'NaamOpleiding': 'Security Management',
  'SoortOpleiding': 'Bachelor',
  'SoortHo': 'HBO',
  'Studiebeschrijving': 'De opleiding security management leert belangrijke skills over criminologie en andere vaardigheden, benodigd voor het politie vak of voor beveiliging',
  'score': 0.84429955}]

In [27]:
import pandas as pd

# Initialize the FAISS index
edu_index = EducationIndex()

# Read the Excel file
excel_path = '/content/drive/MyDrive/DwProfessionsDwEducations/studies.xlsx'
df_studies = pd.read_excel(excel_path)

# Loop over the DataFrame to add each education to the index
for _, row in df_studies.iterrows():
    edu_index.add_education(row.to_dict())

In [34]:
edu_index3 = EducationIndex()
edu_index3.counter = edu_index.counter
edu_index3.education_data = edu_index.education_data
edu_index3.index = edu_index.index

In [35]:
edu_index3.save('/content/drive/MyDrive/DwProfessionsDwEducations/education_index')

In [56]:
education_index = EducationIndex()
education_index.load('/content/drive/MyDrive/DwProfessionsDwEducations/education_index')

In [62]:
education_index.search_by_vector(dw_profession_index2.get_vector_by_id(1))

[{'OpleidingscodeSamen': 70069,
  'NaamOpleiding': 'Tactical Policing',
  'SoortOpleiding': 'Master',
  'SoortHo': 'HBO',
  'Studiebeschrijving': 'De hbo masterstudie Tactical policing (Tactisch Leidinggeven) is een uitstekende voorbereiding op een loopbaan op tactisch leidinggevend niveau. Als teamchef vorm je de schakel tussen het strategische en het operationele niveau. Vanuit een middellange termijn perspectief probeer je, samen met het team waar je leiding aan geeft, antwoorden te vinden op complexe vraagstukken binnen je werkgebied of het organisatieonderdeel waar je verantwoordelijk voor bent. Buiten de organisatie ben je een boegbeeld, binnen een voorbeeld.',
  'score': 0.8556053},
 {'OpleidingscodeSamen': 50422,
  'NaamOpleiding': 'Militaire Bedrijfswetenschappen',
  'SoortOpleiding': 'Bachelor',
  'SoortHo': 'WO',
  'Studiebeschrijving': 'De universitaire bachelorstudie Militaire Bedrijfswetenschappen is gericht op het analyseren en verbeteren van het functioneren van militai

In [61]:
education_index.search(" Timmerman. Timmerlieden knippen, vormen en monteren houten elementen voor de bouw van gebouwen en andere bouwwerken en gebruiken in hun creaties ook materialen als plastic en metaal.")

[{'OpleidingscodeSamen': 34280,
  'NaamOpleiding': 'Werktuigbouwkunde',
  'SoortOpleiding': 'Bachelor',
  'SoortHo': 'HBO',
  'Studiebeschrijving': "De hbo bachelorstudie Werktuigbouwkunde is gericht op het ontwerpen, vervaardigen en onderhouden van producten, apparaten, machines en installaties en op de beheersing van processen. Je leert werktuigen bouwen die het werk van mensen overnemen of vergemakkelijken, variërend van cd-spelers en auto's tot toestellen voor pretparken. Bij de beheersing van processen kun je denken aan de temperatuurregeling in woningen en kantoren en aan de productie van schoon drinkwater. Ook processen die zich afspelen in de motor van een auto of in de pomp van een baggerschip behoren tot dit gebied. Werktuigbouwkunde is dus een breed vakgebied. Je leert hoe je producten ontwerpt en welke productiemiddelen nodig zijn om die te fabriceren. Verder leer je processen regelen en omgaan met installaties en machines die daarvoor nodig zijn. Daarbij besteed je veel aa

In [87]:
import os
import pickle
import pandas as pd

class DwProfessionIndex:
    def __init__(self, dim=1536):
        self.dim = dim
        self.index = faiss.IndexFlatIP(self.dim)
        self.profession_data = {}
        self.counter = 0

    def get_embedding(self, text):
        return get_openai_embedding(text)

    def add_profession(self, profession_entry):
        title = profession_entry.get('title')
        description = profession_entry.get('description')
        nlqf_id = profession_entry.get('nlqf_education_level_id')
        nlqf_text_map = nlqf_text_map = {
        'instroom': 'instroom',
        'NLQF1': 'basiseducatie/vmbo, bb/mbo1',
        'NLQF2': 'vmbo kb, gl, tl/mbo2',
        'NLQF3': 'mbo3',
        'NLQF4': 'mbo4/havo',
        'NLQF4+': 'vwo',
        'NLQF5': 'hbo associate degree',
        'NLQF6': 'hbo bachelor',
        'NLQF7': 'wo master',
        'NLQF8': 'wo doctoraat'
        }

        if title and description and not pd.isna(title) and not pd.isna(description):
            nlqf_text = nlqf_text_map.get(nlqf_id, "onbekend")

            # Concatenate the title, description, and NLQF text
            full_text = f"{title}.\n {description}\nHet minimale opleidingniveau voor {title} is: {nlqf_text}"
            print("adding full text", full_text);
            # Get the embedding
            embedding = self.get_embedding(full_text)

            self.index.add(np.array([embedding]))
            self.profession_data[self.counter] = profession_entry
            self.counter += 1

    def get_vector_by_id(self, profession_id):
        # Search for the index corresponding to the given profession_id
        index = None
        for idx, key in enumerate(self.profession_data.keys()):
            if key == profession_id:
                index = idx
                break

        # If found, reconstruct the vector
        if index is not None:
            vector = np.zeros(self.dim, dtype='float32')
            self.index.reconstruct(index, vector)
            return vector
        else:
            print(f"Profession ID {profession_id} not found.")
            return None


    def save(self, directory_path):
        # Check if the directory exists; create it if it doesn't
        if not os.path.exists(directory_path):
            try:
                os.makedirs(directory_path)
            except Exception as e:
                print(f"Could not create directory: {e}")
                return

        faiss_file_path = os.path.join(directory_path, 'faiss_profession_index.idx')
        data_file_path = os.path.join(directory_path, 'profession_data.pkl')

        faiss.write_index(self.index, faiss_file_path)

        with open(data_file_path, 'wb') as f:
            pickle.dump((self.profession_data, self.counter), f)

    def load(self, directory_path):
        faiss_file_path = os.path.join(directory_path, 'faiss_profession_index.idx')
        data_file_path = os.path.join(directory_path, 'profession_data.pkl')

        self.index = faiss.read_index(faiss_file_path)

        with open(data_file_path, 'rb') as f:
            self.profession_data, self.counter = pickle.load(f)

    def search(self, query_text, k=5):
        query_embedding = self.get_embedding(query_text)
        D, I = self.index.search(np.array([query_embedding]), k)
        results = []
        for idx, score in zip(I[0], D[0]):
            if idx != -1:
                result = copy.deepcopy(self.profession_data[idx])
                result['score'] = score
                results.append(result)
        return results


In [46]:
# Initialize the DwProfessionIndex
dw_profession_index = DwProfessionIndex()

# Read the CSV file (specify the correct path to your uploaded CSV)
csv_path = '/content/drive/MyDrive/DwProfessionsDwEducations/dw_professions_08-09-2023.csv'
df_professions = pd.read_csv(csv_path)

# Loop over the DataFrame to add each profession to the index
for _, row in df_professions.iterrows():
    dw_profession_index.add_profession(row.to_dict())


In [47]:
dw_profession_index.save('/content/drive/MyDrive/DwProfessionsDwEducations/alle_dw_professions_index')

In [48]:
dw_profession_index.search('een beroep waarbij je moet timmeren.')

[{'dw_profession_id': 1720,
  'title': 'timmerman',
  'description': 'Timmerlieden knippen, vormen en monteren houten elementen voor de bouw van gebouwen en andere bouwwerken en gebruiken in hun creaties ook materialen als plastic en metaal.',
  'nlqf_education_level_id': 'NLQF2',
  'score': 0.88116264},
 {'dw_profession_id': 1552,
  'title': 'breeuwer',
  'description': 'Houten ketels drijven eikenhout in de naden tussen planking in dek of romp van houten schepen om ze waterdicht te maken, ze maken voornamelijk gebruik van handgereedschap om scheepslijm te verwarmen en in de naden te duwen, ze kunnen ook henneptouwen en katoenen lijnen in de naden hamer en eroverheen hete pekels uitstrijken.',
  'nlqf_education_level_id': 'NLQF1',
  'score': 0.86945647},
 {'dw_profession_id': 659,
  'title': 'opzichter timmerwerk',
  'description': 'De timmermanagers houden toezicht op de timmerwerkzaamheden in de bouw, wijzen taken toe en nemen snelle beslissingen om problemen op te lossen.',
  'nlqf

In [90]:
dw_profession_index2= DwProfessionIndex()
dw_profession_index2.load('/content/drive/MyDrive/DwProfessionsDwEducations/alle_dw_professions_index')

In [59]:
dw_profession_index2.get_vector_by_id(2)

array([-0.01243078, -0.03078222, -0.00770865, ...,  0.01604605,
        0.01624253,  0.01536491], dtype=float32)

In [91]:
# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame(columns=['dw_profession_id', 'dw_profession_title', 'education_id', 'education_name', 'score'])

# Temporary list to collect data
temp_data = []

# Loop over the dw_professions DataFrame
for _, profession_row in df_professions.iterrows():
    nlqf_level = profession_row['nlqf_education_level_id']

    # Filter out professions with NLQF level below 4
    if nlqf_level not in ['NLQF4', 'NLQF4+', 'NLQF5', 'NLQF6', 'NLQF7', 'NLQF8']:
        continue

    # Get the vector for the current dw_profession
    profession_id = profession_row['dw_profession_id']
    profession_vector = dw_profession_index2.get_vector_by_id(profession_id)

    if profession_vector is None:
        continue

    # Find the 3 most similar educations using the EducationIndex
    similar_educations = education_index.search_by_vector(profession_vector, k=3)



    # Add the results to the result DataFrame
    for match in similar_educations:
        temp_data.append({
            'dw_profession_id': profession_id,
            'dw_profession_title': profession_row['title'],
            'education_id': f"sk:{match['OpleidingscodeSamen']}",
            'education_name': f"{match['NaamOpleiding']} ({match['SoortOpleiding']}, {match['SoortHo']})",
            'score': match['score']
        })

result_df = pd.concat([result_df, pd.DataFrame(temp_data)], ignore_index=True)
# Save the result DataFrame to an Excel file
result_df.to_excel('/content/drive/MyDrive/DwProfessionsDwEducations/dw_profession_to_education_matches.xlsx', index=False)


Profession ID 2548 not found.
Profession ID 2550 not found.
Profession ID 2551 not found.
Profession ID 2552 not found.
Profession ID 2553 not found.
Profession ID 2554 not found.
Profession ID 2555 not found.
Profession ID 2556 not found.
Profession ID 2557 not found.
Profession ID 2558 not found.
Profession ID 2559 not found.
Profession ID 2560 not found.
Profession ID 2561 not found.
Profession ID 2562 not found.
Profession ID 2563 not found.
Profession ID 2564 not found.
Profession ID 2565 not found.
Profession ID 2567 not found.
Profession ID 2568 not found.
Profession ID 2569 not found.
Profession ID 2570 not found.
Profession ID 2571 not found.
Profession ID 2572 not found.
Profession ID 2573 not found.
Profession ID 2574 not found.
Profession ID 2575 not found.
Profession ID 2576 not found.
Profession ID 2577 not found.
Profession ID 2578 not found.
Profession ID 2579 not found.
Profession ID 2580 not found.
Profession ID 2581 not found.
Profession ID 2582 not found.
Profession

In [92]:
# onderstaande dw professions hadden geen tekst.

# Read the CSV file (specify the correct path to your uploaded CSV)
csv_path = '/content/drive/MyDrive/DwProfessionsDwEducations/dw_professions_08-09-2023.csv'
df_professions = pd.read_csv(csv_path)

# Loop over the DataFrame to add each profession to the index
for _, row in df_professions.iterrows():
    dw_profession_id = row['dw_profession_id']  # Replace with the actual column name for the ID, if different

    # Skip professions with ID less than 2548
    if dw_profession_id < 2548:
        continue
    print(row['title'])


technicus waterleidingbedrijf
zweminstructeur
Anesthesioloog
Arbeidstherapeut
Arboverpleegkundige
Arts beleid en advies
Arts infectieziektebestrijding
Arts jeugdgezondheidszorg
Arts maatschappij en gezondheid
Arts medische milieukunde
Arts microbioloog
Arts sociaal medische indicatiestelling en advisering
Arts sociaal-medische dienst
Arts spoedeisende hulp (SEH)
Arts tuberculosebestrijding
Arts voor verstandelijk gehandicapten (AVG)
Basisarts
Bedrijfsarts
begeleider
Brandwondenverpleegkundige
Cardioloog
Cardioloog thoracaal chirurg
casemanager dementie
Chirurg
Consultatief psychiatrisch verpleegkundige
CPA-verpleegkundige / centralist
decubitus verpleegkundige
Decubitus-consulent
Dermatologie verpleegkundige
Dermatoloog
Diabetes verpleegkundige
Dialyseverpleegkundige
Districtsverpleegkundige
Donorarts
endoscopieverpleegkundige
Forensisch arts
Forensisch patholoog
Geriatrisch verpleegkundige
Gezinshuisouder
GG- en GD-verpleegkundige
Gynaecoloog
Helpende instelling
Hoofd kraamzorg
Hoofd 

In [86]:
dw_profession_index2.counter


2552